In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
import pyinform
import scipy
import copy
import pandas as pd
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams["svg.fonttype"] = "none"


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\mizbili\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\mizbili\AppData\Local\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\mizbili\AppData\Local\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start(

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [17]:
#Code to launch network simulation in Male_SHAM, Male_LNIO, Female_SHAM, Female_LNIO conditions


start_time = time.time()



time_tot=2000 #duration of one trial in ms
dt=0.5 # time step
dot_nb=int(time_tot/dt) #nb of dot in one trial


#nb of pyr and FS interneurons in the model
nb_neurons=1000
nb_pyr = int(nb_neurons*80/100)
nb_int = int(nb_neurons*20/100)





#Define parameters of pyramidal neurons (obtained from the fit in Single_neurons_CAd-LIF_models.ipynb)
L_pyr = 30 #µm
diam_pyr = 30 #µm
area_pyr = math.pi*diam_pyr*L_pyr #µm2
#values per area
cm_pyr = 10 #µF/cm-2
gL_pyr = 0.00030165106711688166 #S/cm-2
slope_Na_pyr=5.7
v12mNa_pyr = -35
gAbar_pyr = 0.01 #S/cm-2
#abs model values
C_abs_pyr = (cm_pyr*(area_pyr*pow(10,-8)))*pow(10,6) #pF
gL_abs_pyr = (gL_pyr*(area_pyr*pow(10,-8)))*pow(10,9) #nS
gAbar_abs_pyr = (gAbar_pyr*(area_pyr*pow(10,-8)))*pow(10,9) #nS
EA_pyr = -80.06901458987173
EL_pyr=-78.0
ENa_pyr=50
VD_pyr=-20
VR_pyr=-70.0
VA_pyr = -36.88564167869956
deltaA_pyr = 5.577140058389753
tauA_pyr = 506.6249626721206




#Define parameters of FS interneurons (obtained from the fit in Single_neurons_CAd-LIF_models.ipynb)
L_int = 15 #µm
diam_int = 15 #µm
area_int = math.pi*diam_int*L_int #µm2
#values per area
cm_int = 10 #µF/cm-2
gL_int = 0.001 #S/cm-2
slope_Na_int=5.7
v12mNa_int = -35
gAbar_int = 0.01 #S/cm-2
#abs model values
C_abs_int = (cm_int*(area_int*pow(10,-8)))*pow(10,6) #pF
gL_abs_int = (gL_int*(area_int*pow(10,-8)))*pow(10,9) #nS
gAbar_abs_int = (gAbar_int*(area_int*pow(10,-8)))*pow(10,9) #nS
EA_int = -80.06901458987173
EL_int=-78.0
ENa_int=50
VD_int=-20.0
VR_int=-80.0
VA_int = -36.88564167869956
deltaA_int = 5.577140058389753
tauA_int = 150













#mean and sd of excitatory synapses weight
mean_we = 0.2
sd_we = 1
#mean and sd of inhibitory synapses weight
mean_wi = -30
sd_wi = 4
#connexion probabililty between 2 neurons
proba_connexion=0.1



#Define the scaling factor lambda for the different synapses types (e_e : excit neuron to excit neuron, i_e : inhib neuron to excit neuron, ect...)
#This scaling factor allows to have synaptic weight w in units of mV (see methods)
Rm_pyr = 1/(gL_abs_pyr*pow(10,-9))
taum_pyr = (Rm_pyr * (C_abs_pyr*pow(10,-12)) )*pow(10,3)            
Rm_int = 1/(gL_abs_int*pow(10,-9))
taum_int = (Rm_int * (C_abs_int*pow(10,-12)) )*pow(10,3)
tausyn_e_e = 5
lambda_syn_e_e = (taum_pyr/tausyn_e_e)**(taum_pyr/(taum_pyr-tausyn_e_e))
tausyn_i_e = 10
lambda_syn_i_e = (taum_pyr/tausyn_i_e)**(taum_pyr/(taum_pyr-tausyn_i_e))    
tausyn_e_i = 5
lambda_syn_e_i = (taum_int/tausyn_e_i)**(taum_int/(taum_int-tausyn_e_i))
tausyn_i_i = 10
lambda_syn_i_i = (taum_int/tausyn_i_i)**(taum_int/(taum_int-tausyn_i_i))




#Define dict to save spike nb of pyramidal neurons and FS interneurons at each trial
#dict_spike_nb_pyr={'2_SHAM_Male': [18, 12, 16, 37, 18, 11, 23, 16, 14, 24, 15, 15, 23, 20, 34, 12, 15, 14, 27, 11], '2_LNIO_Male': [30, 26, 34, 189, 38, 29, 106, 24, 24, 49, 30, 33, 53, 28, 57, 24, 30, 65, 54, 89], '2_SHAM_Female': [183, 56, 45, 696, 44, 38, 190, 34, 36, 199, 67, 50, 84, 68, 71, 133, 42, 102, 87, 128], '2_LNIO_Female': [511, 399, 1546, 1916, 997, 939, 1404, 750, 895, 843, 931, 2277, 1090, 555, 810, 1991, 651, 1874, 1317, 951], '10_SHAM_Male': [138, 120, 151, 118, 186, 134, 117, 149, 127, 114, 154, 122, 159, 193, 128, 136, 132, 115, 128, 159], '10_LNIO_Male': [513, 541, 485, 636, 721, 620, 443, 503, 436, 602, 490, 468, 756, 573, 427, 492, 530, 611, 454, 632], '10_SHAM_Female': [788, 799, 915, 925, 1101, 1085, 1280, 918, 902, 946, 1062, 866, 963, 894, 772, 806, 1109, 1026, 1024, 785], '10_LNIO_Female': [2161, 2075, 2012, 1879, 2352, 2526, 2477, 2142, 1907, 2454, 2628, 1991, 2455, 2066, 2553, 1749, 2701, 2574, 2025, 2030], '20_SHAM_Male': [532, 426, 467, 494, 350, 426, 359, 381, 444, 475, 543, 388, 442, 455, 556, 409, 501, 438, 347, 351], '20_LNIO_Male': [1219, 1244, 1169, 1353, 1191, 1333, 1095, 1207, 1099, 1288, 1179, 1187, 1331, 1319, 1355, 1413, 1403, 1234, 1215, 1444], '20_SHAM_Female': [1637, 1737, 1922, 1661, 1632, 1723, 1728, 1635, 1614, 1605, 1752, 1722, 1697, 1562, 2071, 1782, 2084, 1508, 1577, 2090], '20_LNIO_Female': [3018, 3088, 2769, 2769, 3095, 3713, 3096, 3290, 3218, 2701, 3082, 3480, 2849, 2775, 3760, 3221, 2986, 2680, 2678, 3935], '40_SHAM_Male': [1169, 1365, 1261, 1425, 1148, 1226, 1075, 1252, 1170, 1234, 1063, 1285, 1358, 1158, 1167, 1290, 1071, 1210, 1361, 1279], '40_LNIO_Male': [1981, 2321, 2288, 2666, 2170, 2267, 2079, 2249, 2126, 2157, 2084, 2473, 2374, 2258, 2236, 2399, 1971, 2216, 2527, 2301], '40_SHAM_Female': [2585, 3076, 2634, 3067, 2601, 3012, 2686, 2677, 2903, 2605, 2607, 3141, 2904, 2748, 2783, 3187, 2405, 2850, 3073, 2576], '40_LNIO_Female': [4189, 4711, 4154, 4790, 4030, 4207, 4181, 4188, 4931, 3671, 4058, 5168, 4259, 5048, 4272, 4403, 4160, 4574, 4658, 4657], '60_SHAM_Male': [1972, 1893, 1902, 1771, 1854, 1703, 2200, 1759, 1810, 1817, 1815, 1803, 1922, 1745, 2012, 2001, 1689, 1851, 1896, 1946], '60_LNIO_Male': [2940, 3247, 3041, 2887, 2872, 2903, 3035, 2716, 2967, 3143, 2820, 2961, 3112, 2793, 3604, 3139, 2564, 2772, 3007, 3177], '60_SHAM_Female': [3207, 3923, 3927, 3408, 3613, 3561, 3666, 3092, 3473, 3708, 3173, 3351, 3700, 3174, 3727, 4025, 3114, 3355, 3452, 3830], '60_LNIO_Female': [5342, 6016, 5955, 4558, 5073, 4977, 5610, 4485, 5373, 4671, 5097, 5076, 5629, 4647, 5909, 6463, 4135, 4951, 4845, 5546]}
#dict_spike_nb_int={'2_SHAM_Male': [6, 0, 4, 11, 8, 0, 7, 3, 8, 11, 4, 6, 9, 2, 12, 5, 3, 0, 11, 3], '2_LNIO_Male': [8, 3, 8, 62, 14, 5, 31, 6, 12, 18, 9, 10, 19, 6, 22, 8, 7, 10, 18, 27], '2_SHAM_Female': [47, 10, 12, 186, 14, 5, 59, 10, 14, 50, 15, 13, 31, 17, 31, 42, 9, 24, 28, 46], '2_LNIO_Female': [153, 119, 376, 467, 264, 236, 381, 220, 222, 216, 260, 475, 314, 170, 266, 478, 178, 463, 340, 243], '10_SHAM_Male': [50, 25, 41, 34, 52, 46, 28, 48, 43, 33, 42, 30, 41, 59, 52, 34, 34, 38, 36, 62], '10_LNIO_Male': [147, 171, 137, 172, 186, 169, 120, 148, 129, 170, 145, 126, 193, 151, 138, 132, 138, 178, 133, 209], '10_SHAM_Female': [228, 226, 249, 237, 296, 290, 338, 251, 248, 261, 271, 235, 253, 257, 239, 230, 277, 267, 272, 239], '10_LNIO_Female': [546, 547, 519, 525, 564, 607, 637, 551, 490, 604, 614, 498, 604, 523, 639, 471, 628, 601, 531, 566], '20_SHAM_Male': [137, 118, 137, 157, 109, 126, 124, 120, 135, 155, 130, 123, 124, 141, 171, 121, 134, 128, 117, 103], '20_LNIO_Male': [316, 341, 328, 369, 323, 359, 342, 327, 315, 371, 319, 333, 363, 349, 392, 379, 378, 348, 345, 381], '20_SHAM_Female': [435, 481, 489, 485, 429, 435, 467, 434, 443, 440, 447, 437, 428, 426, 541, 473, 501, 412, 441, 481], '20_LNIO_Female': [724, 761, 711, 744, 733, 791, 764, 794, 751, 699, 708, 801, 713, 688, 872, 753, 710, 690, 682, 850], '40_SHAM_Male': [332, 336, 347, 366, 337, 333, 315, 347, 328, 338, 312, 346, 339, 337, 320, 368, 324, 331, 342, 317], '40_LNIO_Male': [552, 570, 568, 596, 569, 560, 554, 582, 557, 554, 543, 602, 575, 584, 553, 641, 526, 546, 617, 563], '40_SHAM_Female': [647, 743, 655, 691, 692, 694, 665, 644, 699, 632, 640, 697, 680, 692, 708, 776, 621, 673, 734, 639], '40_LNIO_Female': [923, 1008, 943, 1005, 944, 932, 965, 933, 1020, 867, 898, 1062, 922, 1078, 966, 1031, 975, 994, 1029, 987], '60_SHAM_Male': [506, 496, 503, 486, 469, 466, 560, 481, 485, 469, 482, 472, 528, 506, 519, 498, 466, 488, 497, 498], '60_LNIO_Male': [726, 771, 767, 705, 679, 697, 767, 696, 724, 765, 702, 696, 749, 735, 817, 728, 675, 727, 741, 761], '60_SHAM_Female': [798, 927, 890, 819, 824, 811, 894, 815, 824, 854, 789, 777, 879, 799, 857, 898, 783, 827, 842, 884], '60_LNIO_Female': [1158, 1258, 1223, 1071, 1096, 1084, 1287, 1080, 1152, 1073, 1134, 1113, 1181, 1061, 1222, 1232, 959, 1126, 1100, 1182]}
dict_spike_nb_pyr={}
dict_spike_nb_int={}


#List of the different external stimulations frequency tested in Hz (the frequency of stimulation represent the frequency of external EPSPs received by one excitatory neuron)
list_freq_external_stim=[0.2,2,10,20,40,60]

#Define the nb of trial for each external stim frequency tested
n_trial = 20

#turn to True to plot rasterplots
plot_rasterplots=False

for freq_external_stim in list_freq_external_stim:
    
    print()
    print()
    print('freq_external_stim : ',freq_external_stim)
    print()
    print()
    
    
    list_spike_nb_pyr_SHAM_Male=[]
    list_spike_nb_pyr_LNIO_Male=[]
    list_spike_nb_pyr_SHAM_Female=[]
    list_spike_nb_pyr_LNIO_Female=[]
    
    list_spike_nb_int_SHAM_Male=[]
    list_spike_nb_int_LNIO_Male=[]
    list_spike_nb_int_SHAM_Female=[]
    list_spike_nb_int_LNIO_Female=[]

    
    
    for trial in range(n_trial):



        print()
        print()
        print('trial :', trial)

        #define synaptic weights between excitatory neurons
        w_pyr_pyr=[]
        for _ in range(nb_pyr):
            list_trans=[]
            for _ in range(nb_pyr):
                w=-10
                while w<=0:
                    w=np.random.lognormal(mean=mean_we, sigma=sd_we)
                list_trans.append(w)
            w_pyr_pyr.append(list_trans)
        for i in range(nb_pyr):
            for j in range(nb_pyr):
                if i==j:
                    w_pyr_pyr[i][j]=0
                else:
                    rand_nb = random.random()
                    if rand_nb>proba_connexion:
                        w_pyr_pyr[i][j]=0
        w_pyr_pyr = np.array(w_pyr_pyr)

        #define synaptic weights between excitatory neurons and inhibitory neurons
        w_pyr_int=[]
        for _ in range(nb_pyr):
            list_trans=[]
            for _ in range(nb_int):
                w=-10
                while w<=0:
                    w=np.random.lognormal(mean=mean_we, sigma=sd_we)
                list_trans.append(w)
            w_pyr_int.append(list_trans)
        for i in range(nb_pyr):
            for j in range(nb_int):
                rand_nb = random.random()
                if rand_nb>proba_connexion:
                    w_pyr_int[i][j]=0
        w_pyr_int = np.array(w_pyr_int)

        #define synaptic weights between inhibitory neurons
        w_int_int=[]
        for _ in range(nb_int):
            list_trans=[]
            for _ in range(nb_int):
                w=10
                while w>=0:
                    w=random.gauss(mean_wi, sd_wi)
                list_trans.append(w)
            w_int_int.append(list_trans)
        for i in range(nb_int):
            for j in range(nb_int):
                if i==j:
                    w_int_int[i][j]=0
                else:
                    rand_nb = random.random()
                    if rand_nb>proba_connexion:
                        w_int_int[i][j]=0
        w_int_int = np.array(w_int_int)              

        #define synaptic weights between inhibitory neurons and excitatory neurons
        w_int_pyr=[]
        for _ in range(nb_int):
            list_trans=[]
            for _ in range(nb_pyr):
                w=10
                while w>=0:
                    w=random.gauss(mean_wi, sd_wi)
                list_trans.append(w)
            w_int_pyr.append(list_trans)
        for i in range(nb_int):
            for j in range(nb_pyr):
                rand_nb = random.random()
                if rand_nb>proba_connexion:
                    w_int_pyr[i][j]=0
        w_int_pyr = np.array(w_int_pyr)

        #Turn external stim frequency into probability of getting one EPSP at a given time step in a given excitatory neuron
        prob_true = freq_external_stim*time_tot/(1000*dot_nb)
        all_presyn_spike_ext_pyr = np.random.choice([True, False], size=(nb_pyr, dot_nb -1), p=[prob_true, 1 - prob_true])

        w_syn_ext=[]
        for i in range(dot_nb -1):
            list_trans=[]
            spike_ext_happening_pyr = all_presyn_spike_ext_pyr[:,i]==True
            for ele in spike_ext_happening_pyr:
                if ele==True:
                    list_trans.append(np.random.lognormal(mean = mean_we, sigma = sd_we))
                else:
                    list_trans.append(0)
            list_trans = np.array(list_trans)
            w_syn_ext.append(list_trans)


        for condition in ['SHAM_Male','LNIO_Male','SHAM_Female','LNIO_Female']:

            print()
            print(condition)
         
            #Define parameters of pyramidal neurons in different conditions (obtained from the fit in Single_neurons_CAd-LIF_models.ipynb)
            if 'SHAM_Male' in condition:
                gNabar_pyr = 0.003 #S/cm-2
                gNabar_int = 0.015 #S/cm-2
            elif 'LNIO_Male' in condition:
                gNabar_pyr = 0.004 #S/cm-2
                gNabar_int = 0.015 #S/cm-2
            elif 'SHAM_Female' in condition:
                gNabar_pyr = 0.0044 #S/cm-2
                gNabar_int = 0.015 #S/cm-2
            elif 'LNIO_Female' in condition:
                gNabar_pyr = 0.0055 #S/cm-2
                gNabar_int = 0.015 #S/cm-2
                
            gNabar_abs_pyr = (gNabar_pyr*(area_pyr*pow(10,-8)))*pow(10,9) #nS
            gNabar_abs_int = (gNabar_int*(area_int*pow(10,-8)))*pow(10,9) #nS
                
            if 'SHAM_Male' in condition:
                deltagA1_pyr = 1.1
                deltagA2_pyr = 0.031
                deltagA3_pyr = 0.0
                deltagA1_int = 0.05
                deltagA2_int = 0.014
                deltagA3_int = 0.004
            elif 'LNIO_Male' in condition:
                deltagA1_pyr = 0.4
                deltagA2_pyr = 0.073
                deltagA3_pyr = 0.0
                deltagA1_int = 0.05
                deltagA2_int = 0.014
                deltagA3_int = 0.004
            elif 'SHAM_Female' in condition:
                deltagA1_pyr = 0.3
                deltagA2_pyr = 0.003
                deltagA3_pyr = 0.006
                deltagA1_int = 0.05
                deltagA2_int = 0.014
                deltagA3_int = 0.004
            elif 'LNIO_Female' in condition:
                deltagA1_pyr = 0.07
                deltagA2_pyr = 0.001
                deltagA3_pyr = 0.0052
                deltagA1_int = 0.05
                deltagA2_int = 0.014
                deltagA3_int = 0.004


            
            #vectors to store vm and gA along the simulation for each neuron
            all_vm_pyr = np.full((nb_pyr, dot_nb),EL_pyr)
            all_vm_int = np.full((nb_int, dot_nb),EL_int)
            all_gA_pyr = np.zeros((nb_pyr, dot_nb))
            all_gA_int = np.zeros((nb_int, dot_nb))


            #lists to store spike time occurence for each neuron
            rasterplots_pyr=[]
            for _ in range(nb_pyr):
                rasterplot=[]
                rasterplots_pyr.append([])
            rasterplots_int=[]
            for _ in range(nb_int):
                rasterplot=[]
                rasterplots_int.append([])

            #vectors to store gsyn along the simulation between each neuronal pair
            all_gsyn_pyr_pyr = np.zeros((nb_pyr, nb_pyr))
            all_gsyn_pyr_int = np.zeros((nb_pyr, nb_int))
            all_gsyn_int_int = np.zeros((nb_int, nb_int))
            all_gsyn_int_pyr = np.zeros((nb_int, nb_pyr))

            #vectors to store external gsyn along the simulation for each neuron
            all_gsyn_ext_pyr = np.zeros((nb_pyr, dot_nb))

            #list to save time during the simulation
            time_list=[0]*dot_nb

            time_simul=0

            #launch the simulation
            for i in range(dot_nb-1):
                
                time_simul=(i+1)*dt
                time_list[i+1]=time_simul

                #---Computations of the different variables for excitatory neurons
                
                #store vm to compute modifications 
                all_vm_soma_prev_pyr = all_vm_pyr[:, i]
                all_gA_soma_prev_pyr = all_gA_pyr[:, i]
                all_gsyn_ext_prev_pyr = all_gsyn_ext_pyr[:,i]

                #compute gA and minf from vm and increment vm from it
                gA_soma_next_pyr = all_gA_soma_prev_pyr + (1/tauA_pyr)*dt*((gAbar_abs_pyr / (1 + np.exp((VA_pyr - all_vm_soma_prev_pyr) / deltaA_pyr))) - all_gA_soma_prev_pyr)
                minf_pyr = 1 / (1 + np.exp(-(all_vm_soma_prev_pyr - v12mNa_pyr) / slope_Na_pyr))
                vm_soma_next_pyr = all_vm_soma_prev_pyr + (dt / C_abs_pyr) * (gL_abs_pyr * (EL_pyr - all_vm_soma_prev_pyr) + gA_soma_next_pyr * (EA_pyr - all_vm_soma_prev_pyr) + gNabar_abs_pyr * minf_pyr * (ENa_pyr - all_vm_soma_prev_pyr) )

                #compute external gsyn and increment vm from it
                gsyn_ext_next_pyr = -all_gsyn_ext_prev_pyr*dt/tausyn_e_e + all_gsyn_ext_prev_pyr
                vm_soma_next_pyr = vm_soma_next_pyr + (1/taum_pyr)*dt*(gsyn_ext_next_pyr)

                #compute gsyn coming from excitatory neurons and increment vm from it
                all_gsyn_pyr_pyr = -all_gsyn_pyr_pyr*dt/tausyn_e_e + all_gsyn_pyr_pyr
                sum_gsyn_pyr_pyr = all_gsyn_pyr_pyr.sum(axis=0)
                vm_soma_next_pyr = vm_soma_next_pyr + (1/taum_pyr)*dt*(sum_gsyn_pyr_pyr)

                #compute gsyn coming from inhibitory neurons and increment vm from it
                all_gsyn_int_pyr = -all_gsyn_int_pyr*dt/tausyn_i_e + all_gsyn_int_pyr
                sum_gsyn_int_pyr = all_gsyn_int_pyr.sum(axis=0)
                vm_soma_next_pyr = vm_soma_next_pyr + (1/taum_pyr)*dt*(sum_gsyn_int_pyr)

                #check if spike is happening, reset vm and increment gA if that is the case
                spike_happening_pyr = vm_soma_next_pyr >= VD_pyr
                vm_soma_next_pyr[spike_happening_pyr] = VR_pyr
                gA_soma_next_pyr[spike_happening_pyr] = gA_soma_next_pyr[spike_happening_pyr] + deltagA1_pyr + deltagA2_pyr*gA_soma_next_pyr[spike_happening_pyr] + deltagA3_pyr*gA_soma_next_pyr[spike_happening_pyr]*gA_soma_next_pyr[spike_happening_pyr]

                #clip Vm to -90 mV hyperpol at minimum
                E_rev_inhib_reached = vm_soma_next_pyr < -90
                vm_soma_next_pyr[E_rev_inhib_reached] = -90

                #increment gsyn for next step if presynaptic spike is happening
                all_gsyn_pyr_pyr[spike_happening_pyr,:] = all_gsyn_pyr_pyr[spike_happening_pyr,:] + w_pyr_pyr[spike_happening_pyr,:]*lambda_syn_e_e
                all_gsyn_pyr_int[spike_happening_pyr,:] = all_gsyn_pyr_int[spike_happening_pyr,:] + w_pyr_int[spike_happening_pyr,:]*lambda_syn_e_i

                #increment external gsyn for next step
                gsyn_ext_next_pyr = gsyn_ext_next_pyr + w_syn_ext[i]*lambda_syn_e_e

                #store spike times in rasterplot lists
                for pyr_nb,bool_spike_happening in enumerate(spike_happening_pyr):
                    if bool_spike_happening==True:
                        rasterplots_pyr[pyr_nb].append(time_simul)

                #store the vm, gA and external gsyn in vectors for the next step
                all_vm_pyr[:, i + 1] = vm_soma_next_pyr
                all_gA_pyr[:, i + 1] = gA_soma_next_pyr
                all_gsyn_ext_pyr[:, i + 1] = gsyn_ext_next_pyr



                #---Computations of the different variables for inhibitory neurons

                #store vm to compute modifications
                all_vm_soma_prev_int = all_vm_int[:, i]
                all_gA_soma_prev_int = all_gA_int[:, i]

                #compute gA and minf from vm and increment vm from it
                gA_soma_next_int = all_gA_soma_prev_int + (1/tauA_int)*dt*((gAbar_abs_int / (1 + np.exp((VA_int - all_vm_soma_prev_int) / deltaA_int))) - all_gA_soma_prev_int)
                minf_int = 1 / (1 + np.exp(-(all_vm_soma_prev_int - v12mNa_int) / slope_Na_int))
                vm_soma_next_int = all_vm_soma_prev_int + (dt / C_abs_int) * (gL_abs_int * (EL_int - all_vm_soma_prev_int) + gA_soma_next_int * (EA_int - all_vm_soma_prev_int) + gNabar_abs_int * minf_int * (ENa_int - all_vm_soma_prev_int) )

                #compute gsyn coming from inhibitory neurons and increment vm from it
                all_gsyn_int_int = -all_gsyn_int_int*dt/tausyn_i_i + all_gsyn_int_int
                sum_gsyn_int_int = all_gsyn_int_int.sum(axis=0)
                vm_soma_next_int = vm_soma_next_int + (1/taum_int)*dt*(sum_gsyn_int_int)

                #compute gsyn coming from excitatory neurons and increment vm from it
                all_gsyn_pyr_int = -all_gsyn_pyr_int*dt/tausyn_e_i + all_gsyn_pyr_int
                sum_gsyn_pyr_int = all_gsyn_pyr_int.sum(axis=0)
                vm_soma_next_int = vm_soma_next_int + (1/taum_int)*dt*(sum_gsyn_pyr_int)

                #check if spike is happening, reset vm and increment gA if that is the case
                spike_happening_int = vm_soma_next_int >= VD_int
                vm_soma_next_int[spike_happening_int] = VR_int
                gA_soma_next_int[spike_happening_int] = gA_soma_next_int[spike_happening_int] + deltagA1_int + deltagA2_int*gA_soma_next_int[spike_happening_int] + deltagA3_int*gA_soma_next_int[spike_happening_int]*gA_soma_next_int[spike_happening_int]

                #clip Vm to -90 mV hyperpol at minimum
                E_rev_inhib_reached = vm_soma_next_int < -90
                vm_soma_next_int[E_rev_inhib_reached] = -90

                #increment gsyn for next step if presynaptic spike is happening
                all_gsyn_int_int[spike_happening_int,:] = all_gsyn_int_int[spike_happening_int,:] + w_int_int[spike_happening_int,:]*lambda_syn_i_i
                all_gsyn_int_pyr[spike_happening_int,:] = all_gsyn_int_pyr[spike_happening_int,:] + w_int_pyr[spike_happening_int,:]*lambda_syn_i_e

                #store spike times in rasterplot lists
                for int_nb,bool_spike_happening in enumerate(spike_happening_int):
                    if bool_spike_happening==True:
                        rasterplots_int[int_nb].append(time_simul)

                #store the vm, gA and external gsyn in vectors for the next step
                all_vm_int[:, i + 1] = vm_soma_next_int
                all_gA_int[:, i + 1] = gA_soma_next_int




            #Uncomment to plot voltage traces
            '''
            if 'SHAM_Male' in condition:
                for vm_soma in all_vm_pyr:
                    plt.plot(time_list,vm_soma,'k')
            elif 'LNIO_Male' in condition:
                for vm_soma in all_vm_pyr:
                    plt.plot(time_list,vm_soma,'r')
            elif 'SHAM_Female' in condition:
                for vm_soma in all_vm_pyr:
                    plt.plot(time_list,vm_soma,'dimgrey')
            elif 'LNIO_Female' in condition:
                for vm_soma in all_vm_pyr:
                    plt.plot(time_list,vm_soma,'darkorange')
            for vm_soma in all_vm_int:
                plt.plot(time_list,vm_soma,'g')
            plt.show()
            '''


            #plot rasterplots and save spike nb in pyr and interneurons that occured during the simulations
            nb_spike_pyr=0
            for pyr_nb,rasterplot in enumerate(rasterplots_pyr):
                for spike_time in rasterplot:
                    if plot_rasterplots==True:
                        if 'SHAM_Male' in condition:
                            plt.plot(spike_time,pyr_nb,'ko',ms=3)
                        elif 'LNIO_Male' in condition:
                            plt.plot(spike_time,pyr_nb,'ro',ms=3)
                        elif 'SHAM_Female' in condition:
                            plt.plot(spike_time,pyr_nb,'o',color='dimgrey',ms=3)
                        elif 'LNIO_Female' in condition:
                            plt.plot(spike_time,pyr_nb,'o',color='darkorange',ms=3)
                    nb_spike_pyr=nb_spike_pyr+1

            nb_spike_int=0
            for int_nb,rasterplot in enumerate(rasterplots_int):
                for spike_time in rasterplot:
                    if plot_rasterplots==True:
                        plt.plot(spike_time,nb_pyr+int_nb,'go',ms=3)
                    nb_spike_int=nb_spike_int+1

            if plot_rasterplots==True:
                plt.xlim(0,time_tot)
                plt.savefig('Rasterplot_Model_'+condition+'.svg') 
                plt.show()
            
            print('nb_spike_tot ',nb_spike_pyr+nb_spike_int)
            print('nb_spike_pyr ',nb_spike_pyr)
            print('nb_spike_int ',nb_spike_int)
            
            if condition=='SHAM_Male':
                list_spike_nb_pyr_SHAM_Male.append(nb_spike_pyr)
                list_spike_nb_int_SHAM_Male.append(nb_spike_int)
            elif condition=='LNIO_Male':
                list_spike_nb_pyr_LNIO_Male.append(nb_spike_pyr)
                list_spike_nb_int_LNIO_Male.append(nb_spike_int)
            elif condition=='SHAM_Female':
                list_spike_nb_pyr_SHAM_Female.append(nb_spike_pyr)
                list_spike_nb_int_SHAM_Female.append(nb_spike_int)
            elif condition=='LNIO_Female':
                list_spike_nb_pyr_LNIO_Female.append(nb_spike_pyr)
                list_spike_nb_int_LNIO_Female.append(nb_spike_int)


            time_comput = time.time() - start_time
            if time_comput<60:
                print("time of computation : ", time_comput," seconds")
            else:
                print("time of computation : ", time_comput/60," minutes")
    
    
    
    #save in dictionnaries the spike nb in the different trials    
    dict_spike_nb_pyr[str(freq_external_stim)+'_SHAM_Male']=list_spike_nb_pyr_SHAM_Male
    dict_spike_nb_pyr[str(freq_external_stim)+'_LNIO_Male']=list_spike_nb_pyr_LNIO_Male
    dict_spike_nb_pyr[str(freq_external_stim)+'_SHAM_Female']=list_spike_nb_pyr_SHAM_Female
    dict_spike_nb_pyr[str(freq_external_stim)+'_LNIO_Female']=list_spike_nb_pyr_LNIO_Female

    dict_spike_nb_int[str(freq_external_stim)+'_SHAM_Male']=list_spike_nb_int_SHAM_Male
    dict_spike_nb_int[str(freq_external_stim)+'_LNIO_Male']=list_spike_nb_int_LNIO_Male
    dict_spike_nb_int[str(freq_external_stim)+'_SHAM_Female']=list_spike_nb_int_SHAM_Female
    dict_spike_nb_int[str(freq_external_stim)+'_LNIO_Female']=list_spike_nb_int_LNIO_Female
        

#Uncomment to save results in excel file
'''
df_spike_nb_pyr=pd.DataFrame.from_dict(dict_spike_nb_pyr)
df_spike_nb_int=pd.DataFrame.from_dict(dict_spike_nb_int)
with pd.ExcelWriter('Network_results.xlsx') as writer:  
    df_spike_nb_pyr.to_excel(writer, sheet_name='spike_nb_pyr')
    df_spike_nb_int.to_excel(writer, sheet_name='spike_nb_int')
'''



freq_external_stim :  0.2




trial : 0

SHAM_Male
nb_spike_tot  2
nb_spike_pyr  1
nb_spike_int  1
time of computation :  55.83217263221741  seconds

LNIO_Male
nb_spike_tot  2
nb_spike_pyr  1
nb_spike_int  1
time of computation :  1.8047709107398986  minutes

SHAM_Female
nb_spike_tot  2
nb_spike_pyr  1
nb_spike_int  1
time of computation :  2.622667415936788  minutes

LNIO_Female
nb_spike_tot  4
nb_spike_pyr  2
nb_spike_int  2
time of computation :  3.4302958488464355  minutes


trial : 1

SHAM_Male
nb_spike_tot  0
nb_spike_pyr  0
nb_spike_int  0
time of computation :  4.318047932783762  minutes

LNIO_Male
nb_spike_tot  0
nb_spike_pyr  0
nb_spike_int  0
time of computation :  5.132554980119069  minutes

SHAM_Female
nb_spike_tot  0
nb_spike_pyr  0
nb_spike_int  0
time of computation :  5.939505195617675  minutes

LNIO_Female
nb_spike_tot  9
nb_spike_pyr  5
nb_spike_int  4
time of computation :  6.738082313537598  minutes


trial : 2

SHAM_Male
nb_spike_tot  4
nb_spike_pyr  3
nb_spike_

PermissionError: [Errno 13] Permission denied: 'Network_results.xlsx'